# SQL Query Practice

Data source: https://www.kaggle.com/ananta/credit-card-data

## Create and connect to a sqlite  database
If we attempt to connect to a database that does not exist, it will be created. Always close your connection when you are finished.

In [1]:
import sqlite3
from sqlite3 import Error

cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

print(sqlite3.version)

cur.close()
cnx.close()


/mnt/c/Users/Nikki/Sites/Data-Science-Practice
2.6.0


## Import tables with pandas
Using pandas dataframes prevents us from having to read the csv file and iterate through rows

In [2]:
import pandas as pd

# Read csv to dataframes

df_cards = pd.read_csv('data/credit_card_data/CardBase.csv')
df_customers = pd.read_csv('data/credit_card_data/CustomerBase.csv')
df_transactions = pd.read_csv('data/credit_card_data/TransactionBase.csv')
df_frauds = pd.read_csv('data/credit_card_data/FraudBase.csv')


# View table info and first 5 rows of each table
newline = '\n'
print(df_cards.info())
display(df_cards.head())
print(newline)

print(df_customers.info())
display(df_customers.head())
print(newline)

print(df_transactions.info())
display(df_transactions.head())
print(newline)

print(df_frauds.info())
display(df_frauds.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Card_Number   500 non-null    object
 1   Card_Family   500 non-null    object
 2   Credit_Limit  500 non-null    int64 
 3   Cust_ID       500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5674 entries, 0 to 5673
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Cust_ID                 5674 non-null   object
 1   Age                     5674 non-null   int64 
 2   Customer_Segment        5674 non-null   object
 3   Customer_Vintage_Group  5674 non-null   object
dtypes: int64(1), object(3)
memory usage: 177.4+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #

,Card_Number,Card_Family,Credit_Limit,Cust_ID
0,8638-5407-3631-8196,Premium,530000,CC67088
1,7106-4239-7093-1515,Gold,18000,CC12076
2,6492-5655-8241-3530,Premium,596000,CC97173
3,2868-5606-5152-5706,Gold,27000,CC55858
4,1438-6906-2509-8219,Platinum,142000,CC90518


,Cust_ID,Age,Customer_Segment,Customer_Vintage_Group
0,CC25034,35,Diamond,VG1
1,CC59625,22,Diamond,VG1
2,CC69314,33,Diamond,VG1
3,CC67036,46,Diamond,VG1
4,CC25597,44,Diamond,VG1


,Transaction_ID,Transaction_Date,Credit_Card_ID,Transaction_Value,Transaction_Segment
0,CTID28830551,24-Apr-16,1629-9566-3285-2123,23649,SEG25
1,CTID45504917,11-Feb-16,3697-6001-4909-5350,26726,SEG16
2,CTID47312290,1-Nov-16,5864-4475-3659-1440,22012,SEG14
3,CTID25637718,28-Jan-16,5991-4421-8476-3804,37637,SEG17
4,CTID66743960,17-Mar-16,1893-8853-9900-8478,5113,SEG14


,Transaction_ID,Fraud_Flag
0,CTID50558449,1
1,CTID55936882,1
2,CTID63762180,1
3,CTID76723439,1
4,CTID21246201,1


## Drop and create sqlite tables

We are going to check if each table exists. If not, we will create it.
The code below uses 'string formatting' to make string concatenation easier

In [3]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Check if table exists in sqlite database

def table_exists(table_name):  
    #get the count of tables with the name
    result = cur.execute('SELECT count(name) FROM sqlite_master WHERE type="table" AND name="%s"' % table_name)

    #if the count is 1, then table exists
    if cur.fetchone()[0]==1 :
        print('%s exists' % table_name)
        return True
    
    print('%s does not exist' % table_name)
    return False
 
    
# DROP TABLE

def drop_table(table_name):
    print('drop %s' % table_name)
    sql = 'DROP TABLE %s' % table_name
    cur.execute(sql)
    
    
# Put table info into a list of tuples to keep code "DRY"

tables = [('cards', df_cards), ('customers', df_customers), ('transactions', df_transactions), ('frauds', df_frauds)]

for (table_name, df) in tables:
#     if table_exists(table_name):
#         drop_table(table_name)
      
    # CREATE table
    if not table_exists(table_name):
        print('CREATE %s' % table_name)   
        df.to_sql(name=table_name, con=cnx)
   
cur.close()   
cnx.close()

cards exists
customers exists
transactions exists
frauds exists


## Read the data

To understand what data we are working with, we will connect to a sqlite database, list the table names, list the column names, and preview data for each table.

We already saw this information in our dataframes above, but let's do it this way as as an exercise.

In [4]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

newline_tab =  '\n  '

# Print table names

result = cur.execute('SELECT name FROM sqlite_master WHERE type="table"').fetchall()
table_names = list(zip(*result))[0]
print ('Tables:', newline_tab.join(table_names), sep = newline_tab)
print(newline)

# Print column names

for table_name in table_names:
    result = cur.execute('PRAGMA table_info("%s")' % table_name).fetchall()
    print ('Columns for %s:' % table_name)
    print(print(*result, sep=newline))
    print(newline)
    
    
    print ('Columns for %s:' % table_name)
    rows = cur.execute('SELECT * from %s LIMIT 5' % table_name)
    for row in rows:
        print(row)
    print(newline)
   
cur.close()  
cnx.close()

Tables:
  cards
  customers
  transactions
  frauds


Columns for cards:
(0, 'index', 'INTEGER', 0, None, 0)
(1, 'Card_Number', 'TEXT', 0, None, 0)
(2, 'Card_Family', 'TEXT', 0, None, 0)
(3, 'Credit_Limit', 'INTEGER', 0, None, 0)
(4, 'Cust_ID', 'TEXT', 0, None, 0)
None


Columns for cards:
(0, '8638-5407-3631-8196', 'Premium', 530000, 'CC67088')
(1, '7106-4239-7093-1515', 'Gold', 18000, 'CC12076')
(2, '6492-5655-8241-3530', 'Premium', 596000, 'CC97173')
(3, '2868-5606-5152-5706', 'Gold', 27000, 'CC55858')
(4, '1438-6906-2509-8219', 'Platinum', 142000, 'CC90518')


Columns for customers:
(0, 'index', 'INTEGER', 0, None, 0)
(1, 'Cust_ID', 'TEXT', 0, None, 0)
(2, 'Age', 'INTEGER', 0, None, 0)
(3, 'Customer_Segment', 'TEXT', 0, None, 0)
(4, 'Customer_Vintage_Group', 'TEXT', 0, None, 0)
None


Columns for customers:
(0, 'CC25034', 35, 'Diamond', 'VG1')
(1, 'CC59625', 22, 'Diamond', 'VG1')
(2, 'CC69314', 33, 'Diamond', 'VG1')
(3, 'CC67036', 46, 'Diamond', 'VG1')
(4, 'CC25597', 44, 'Diamond',

## Join Types

Outer joins return every row from at least one table.

#### Left outer joins 
All rows from the left table with any matches from the right table. If no matches, null is returned for each column of the right table.

This is the join type you will use 99% of the time. It is great for one-to-one and one-to-many relationships.

In [5]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Show me all of the data on customers and their credit cards

cur.execute('''SELECT * 
               FROM customers 
                 LEFT JOIN cards ON customers.CUST_ID = cards.CUST_ID''')
customer_cards = pd.DataFrame(cur.fetchall())
customer_cards.columns = [x[0] for x in cur.description]
display(customer_cards)

# Possible next steps: Group by customer and count number of cards


# ----- RIGHT JOIN (not supported by sqlite) -----

# Request: I want a list of transactions preceded by a flag if they are fraudulent
# Possible next steps: Group by credit card number and get the sum of fraudulent transaction amounts

# transactions_flagged = pd.read_sql('''SELECT frauds.Fraud_Flag, transactions.*
#                    FROM frauds 
#                      RIGHT JOIN transactions ON frauds.Transaction_ID = cards.Transaction_ID''',
#                con=cnx)
transactions_flagged = pd.read_sql(
    '''SELECT frauds.Fraud_Flag, transactions.*
       FROM transactions 
         LEFT JOIN frauds ON frauds.Transaction_ID = transactions.Transaction_ID''',
    con=cnx, index_col='index')
display(transactions_flagged)

   
cur.close()  
cnx.close()

,index,Cust_ID,Age,Customer_Segment,Customer_Vintage_Group,index,Card_Number,Card_Family,Credit_Limit,Cust_ID
0,0,CC25034,35,Diamond,VG1,NaN,None,None,NaN,None
1,1,CC59625,22,Diamond,VG1,NaN,None,None,NaN,None
2,2,CC69314,33,Diamond,VG1,NaN,None,None,NaN,None
3,3,CC67036,46,Diamond,VG1,NaN,None,None,NaN,None
4,4,CC25597,44,Diamond,VG1,NaN,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...
5687,5669,CC53430,27,Platinum,VG2,NaN,None,None,NaN,None
5688,5670,CC31823,33,Platinum,VG2,NaN,None,None,NaN,None
5689,5671,CC15335,28,Platinum,VG2,NaN,None,None,NaN,None
5690,5672,CC81556,26,Platinum,VG2,NaN,None,None,NaN,None


,Fraud_Flag,Transaction_ID,Transaction_Date,Credit_Card_ID,Transaction_Value,Transaction_Segment
index,,,,,,
0,NaN,CTID28830551,24-Apr-16,1629-9566-3285-2123,23649,SEG25
1,NaN,CTID45504917,11-Feb-16,3697-6001-4909-5350,26726,SEG16
2,NaN,CTID47312290,1-Nov-16,5864-4475-3659-1440,22012,SEG14
3,NaN,CTID25637718,28-Jan-16,5991-4421-8476-3804,37637,SEG17
4,NaN,CTID66743960,17-Mar-16,1893-8853-9900-8478,5113,SEG14
...,...,...,...,...,...,...
9995,NaN,CTID25037573,10-Jun-16,9157-2802-8374-1145,44280,SEG18
9996,NaN,CTID43832298,23-May-16,7416-4529-6690-5703,27163,SEG12
9997,NaN,CTID56532072,6-Aug-16,6699-2639-4522-6219,36424,SEG13



#### Right outer joins
All rows from the right table with any matches from the left table. If no matches, null is returned for each column of the left table.

These queries can always be written as the opposite left join. You might want to use a right join to makes your query easier to read and understand.

In [6]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: I want a list of transactions preceded by a flag if they are fraudulent

# RIGHT JOIN below is not supported by sqlite

# transactions_flagged = pd.read_sql('''SELECT frauds.Fraud_Flag, transactions.*
#                    FROM frauds 
#                      RIGHT JOIN transactions ON frauds.Transaction_ID = cards.Transaction_ID''',
#                con=cnx)

# Equivalent supported query

transactions_flagged = pd.read_sql(
    '''SELECT frauds.Fraud_Flag, transactions.*
       FROM transactions 
         LEFT JOIN frauds ON frauds.Transaction_ID = transactions.Transaction_ID''',
    con=cnx, index_col='index')
display(transactions_flagged)


# Possible next steps: Get the total of fraudulent transaction amounts for every credit card

fraud_amounts_by_card = pd.read_sql(
    '''SELECT SUM(
         CASE WHEN frauds.Fraud_Flag = 1
           THEN transactions.Transaction_Value 
           ELSE 0 END
        ) AS Fraud_Total, transactions.Credit_Card_ID
       FROM transactions 
         LEFT JOIN frauds ON frauds.Transaction_ID = transactions.Transaction_ID
       GROUP BY transactions.Credit_Card_ID''',
   con=cnx)
display(fraud_amounts_by_card)

   
cur.close()  
cnx.close()

,Fraud_Flag,Transaction_ID,Transaction_Date,Credit_Card_ID,Transaction_Value,Transaction_Segment
index,,,,,,
0,NaN,CTID28830551,24-Apr-16,1629-9566-3285-2123,23649,SEG25
1,NaN,CTID45504917,11-Feb-16,3697-6001-4909-5350,26726,SEG16
2,NaN,CTID47312290,1-Nov-16,5864-4475-3659-1440,22012,SEG14
3,NaN,CTID25637718,28-Jan-16,5991-4421-8476-3804,37637,SEG17
4,NaN,CTID66743960,17-Mar-16,1893-8853-9900-8478,5113,SEG14
...,...,...,...,...,...,...
9995,NaN,CTID25037573,10-Jun-16,9157-2802-8374-1145,44280,SEG18
9996,NaN,CTID43832298,23-May-16,7416-4529-6690-5703,27163,SEG12
9997,NaN,CTID56532072,6-Aug-16,6699-2639-4522-6219,36424,SEG13


,Fraud_Total,Credit_Card_ID
0,0,1113-9175-3253-8426
1,0,1123-9777-7335-9167
2,0,1127-9633-2269-3119
3,0,1159-7886-1385-5540
4,22063,1175-3754-1370-5515
...,...,...
495,0,9933-8699-5268-4345
496,0,9946-6939-3033-9593
497,0,9961-2869-2603-3317
498,0,9991-4727-7710-2269


#### Full outer joins
All rows from the left table and the right table. If no matches, null is returned for each column of the left or right table as needed.

This is a great query to see if you have missing data so you can address the issue.

In [7]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Make sure we don't have any orphaned data between customers and credit cards

# OUTER JOIN is not supported by sqlite

# customers_cards = pd.read_sql(
#     '''SELECT *
#        FROM customers 
#          FULL OUTER JOIN cards ON customers.Customer_ID = cards.Customer_ID''',
#     con=cnx)

# Equivalent supported query

customers_cards = pd.read_sql(
    '''SELECT customers.*, cards.*
       FROM customers
         LEFT JOIN cards ON customers.Cust_ID = cards.Cust_ID
       UNION ALL
       SELECT customers.*, cards.* 
       FROM cards
         LEFT JOIN customers ON customers.Cust_ID = cards.Cust_ID
       WHERE customers.Cust_ID IS NULL''',
    con=cnx)
display(customers_cards)


# Possible next steps: Display only rows where data is missing

rows_missing_data = pd.read_sql(
    '''SELECT * FROM (
         SELECT customers.*, cards.*
         FROM customers
           LEFT JOIN cards ON customers.Cust_ID = cards.Cust_ID
         UNION ALL
         SELECT customers.*, cards.* 
         FROM cards
           LEFT JOIN customers ON customers.Cust_ID = cards.Cust_ID
         WHERE customers.Cust_ID IS NULL) as joined_table
       WHERE Cust_ID is NULL OR Card_Number is NULL''',
    con=cnx)
display(rows_missing_data)

# Note: If you only want rows with missing data from one table, a simple left join will suffice. 
# For example to show only customers with no card data

cur.close()  
cnx.close()

,index,Cust_ID,Age,Customer_Segment,Customer_Vintage_Group,index,Card_Number,Card_Family,Credit_Limit,Cust_ID
0,0,CC25034,35,Diamond,VG1,NaN,None,None,NaN,None
1,1,CC59625,22,Diamond,VG1,NaN,None,None,NaN,None
2,2,CC69314,33,Diamond,VG1,NaN,None,None,NaN,None
3,3,CC67036,46,Diamond,VG1,NaN,None,None,NaN,None
4,4,CC25597,44,Diamond,VG1,NaN,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...
5687,5669,CC53430,27,Platinum,VG2,NaN,None,None,NaN,None
5688,5670,CC31823,33,Platinum,VG2,NaN,None,None,NaN,None
5689,5671,CC15335,28,Platinum,VG2,NaN,None,None,NaN,None
5690,5672,CC81556,26,Platinum,VG2,NaN,None,None,NaN,None


,index,Cust_ID,Age,Customer_Segment,Customer_Vintage_Group,index:1,Card_Number,Card_Family,Credit_Limit,Cust_ID:1
0,0,CC25034,35,Diamond,VG1,None,None,None,None,None
1,1,CC59625,22,Diamond,VG1,None,None,None,None,None
2,2,CC69314,33,Diamond,VG1,None,None,None,None,None
3,3,CC67036,46,Diamond,VG1,None,None,None,None,None
4,4,CC25597,44,Diamond,VG1,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
5187,5669,CC53430,27,Platinum,VG2,None,None,None,None,None
5188,5670,CC31823,33,Platinum,VG2,None,None,None,None,None
5189,5671,CC15335,28,Platinum,VG2,None,None,None,None,None
5190,5672,CC81556,26,Platinum,VG2,None,None,None,None,None


#### Inner joins
Show only rows with matches from both tables

Inner join can be replaced with a left join with where clauses to filter out rows without matches.

In [8]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Show me all fraudulent transactions

fraudulent_transactions = pd.read_sql(
    '''SELECT transactions.* 
       FROM transactions 
         INNER JOIN frauds ON transactions.Transaction_ID = frauds.Transaction_ID''',
    con=cnx)
display(fraudulent_transactions)

# Possible next steps: Show all the customers with fraudulent transactions

defrauded_customers = pd.read_sql(
    '''SELECT DISTINCT customers.*
       FROM transactions
         INNER JOIN frauds ON transactions.Transaction_ID = frauds.Transaction_ID
         INNER JOIN cards ON cards.Card_Number = transactions.Credit_Card_ID
         INNER JOIN customers ON customers.Cust_ID = cards.Cust_ID''',
    con=cnx)
display(defrauded_customers)  

   
cur.close()  
cnx.close()

,index,Transaction_ID,Transaction_Date,Credit_Card_ID,Transaction_Value,Transaction_Segment
0,102,CTID50558449,6-May-16,4246-1369-3659-8804,6984,SEG23
1,219,CTID55936882,29-Nov-16,1336-9200-1264-2551,34367,SEG21
2,336,CTID63762180,5-Dec-16,8528-6154-7390-5081,44550,SEG15
3,494,CTID76723439,15-Sep-16,7908-2695-7391-7499,48275,SEG16
4,560,CTID21246201,29-Feb-16,2524-4184-5908-6750,35751,SEG25
...,...,...,...,...,...,...
104,9841,CTID57993591,2-Apr-16,6697-9358-9213-4896,31486,SEG24
105,9881,CTID91108283,17-Jul-16,9207-1270-6690-4905,36706,SEG11
106,9933,CTID30494187,31-Dec-16,9018-5320-5729-5393,14586,SEG20
107,9973,CTID51301522,28-Jan-16,3620-5235-2101-3391,22354,SEG23


,index,Cust_ID,Age,Customer_Segment,Customer_Vintage_Group
0,143,CC35402,26,Diamond,VG1
1,173,CC42497,23,Diamond,VG1
2,209,CC41731,35,Diamond,VG1
3,221,CC93075,32,Diamond,VG1
4,235,CC64111,41,Diamond,VG1
...,...,...,...,...,...
92,5476,CC87034,36,Platinum,VG2
93,5534,CC87270,28,Platinum,VG2
94,5538,CC81000,25,Platinum,VG2
95,5548,CC66351,22,Platinum,VG2


In [9]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Show me all fraudulent transactions

fraudulent_transactions = pd.read_sql(
    '''SELECT transactions.* 
       FROM transactions 
         INNER JOIN frauds ON transactions.Transaction_ID = frauds.Transaction_ID''',
    con=cnx)
display(fraudulent_transactions)

# Possible next steps: Show all the customers with fraudulent transactions

defrauded_customers = pd.read_sql(
    '''SELECT DISTINCT customers.*
       FROM transactions
         INNER JOIN frauds ON transactions.Transaction_ID = frauds.Transaction_ID
         INNER JOIN cards ON cards.Card_Number = transactions.Credit_Card_ID
         INNER JOIN customers ON customers.Cust_ID = cards.Cust_ID''',
    con=cnx)
display(defrauded_customers)  

   
cur.close()  
cnx.close()

OperationalError: unable to open database file

## Join Types

Outer joins return every row from at least one table.

#### Left outer joins 
All rows from the left table with any matches from the right table. If no matches, null is returned for each column of the right table.

This is the join type you will use 99% of the time. It is great for one-to-one and one-to-many relationships.

In [ ]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Show me all of the data on customers and their credit cards

cur.execute('''SELECT * 
               FROM customers 
                 LEFT JOIN cards ON customers.CUST_ID = cards.CUST_ID''')
customer_cards = pd.DataFrame(cur.fetchall())
customer_cards.columns = [x[0] for x in cur.description]
display(customer_cards)

# Possible next steps: Group by customer and count number of cards


# ----- RIGHT JOIN (not supported by sqlite) -----

# Request: I want a list of transactions preceded by a flag if they are fraudulent
# Possible next steps: Group by credit card number and get the sum of fraudulent transaction amounts

# transactions_flagged = pd.read_sql('''SELECT frauds.Fraud_Flag, transactions.*
#                    FROM frauds 
#                      RIGHT JOIN transactions ON frauds.Transaction_ID = cards.Transaction_ID''',
#                con=cnx)
transactions_flagged = pd.read_sql(
    '''SELECT frauds.Fraud_Flag, transactions.*
       FROM transactions 
         LEFT JOIN frauds ON frauds.Transaction_ID = transactions.Transaction_ID''',
    con=cnx, index_col='index')
display(transactions_flagged)

   
cur.close()  
cnx.close()


#### Right outer joins
All rows from the right table with any matches from the left table. If no matches, null is returned for each column of the left table.

These queries can always be written as the opposite left join. You might want to use a right join to makes your query easier to read and understand.

In [ ]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: I want a list of transactions preceded by a flag if they are fraudulent

# RIGHT JOIN below is not supported by sqlite

# transactions_flagged = pd.read_sql('''SELECT frauds.Fraud_Flag, transactions.*
#                    FROM frauds 
#                      RIGHT JOIN transactions ON frauds.Transaction_ID = cards.Transaction_ID''',
#                con=cnx)

# Equivalent supported query

transactions_flagged = pd.read_sql(
    '''SELECT frauds.Fraud_Flag, transactions.*
       FROM transactions 
         LEFT JOIN frauds ON frauds.Transaction_ID = transactions.Transaction_ID''',
    con=cnx, index_col='index')
display(transactions_flagged)


# Possible next steps: Get the total of fraudulent transaction amounts for every credit card

fraud_amounts_by_card = pd.read_sql(
    '''SELECT SUM(
         CASE WHEN frauds.Fraud_Flag = 1
           THEN transactions.Transaction_Value 
           ELSE 0 END
        ) AS Fraud_Total, transactions.Credit_Card_ID
       FROM transactions 
         LEFT JOIN frauds ON frauds.Transaction_ID = transactions.Transaction_ID
       GROUP BY transactions.Credit_Card_ID''',
   con=cnx)
display(fraud_amounts_by_card)

   
cur.close()  
cnx.close()

#### Full outer joins
All rows from the left table and the right table. If no matches, null is returned for each column of the left or right table as needed.

This is a great query to see if you have missing data so you can address the issue.

In [ ]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Make sure we don't have any orphaned data between customers and credit cards

# OUTER JOIN is not supported by sqlite

# customers_cards = pd.read_sql(
#     '''SELECT *
#        FROM customers 
#          FULL OUTER JOIN cards ON customers.Customer_ID = cards.Customer_ID''',
#     con=cnx)

# Equivalent supported query

customers_cards = pd.read_sql(
    '''SELECT customers.*, cards.*
       FROM customers
         LEFT JOIN cards ON customers.Cust_ID = cards.Cust_ID
       UNION ALL
       SELECT customers.*, cards.* 
       FROM cards
         LEFT JOIN customers ON customers.Cust_ID = cards.Cust_ID
       WHERE customers.Cust_ID IS NULL''',
    con=cnx)
display(customers_cards)


# Possible next steps: Display only rows where data is missing

rows_missing_data = pd.read_sql(
    '''SELECT * FROM (
         SELECT customers.*, cards.*
         FROM customers
           LEFT JOIN cards ON customers.Cust_ID = cards.Cust_ID
         UNION ALL
         SELECT customers.*, cards.* 
         FROM cards
           LEFT JOIN customers ON customers.Cust_ID = cards.Cust_ID
         WHERE customers.Cust_ID IS NULL) as joined_table
       WHERE Cust_ID is NULL OR Card_Number is NULL''',
    con=cnx)
display(rows_missing_data)

# Note: If you only want rows with missing data from one table, a simple left join will suffice. 
# For example to show only customers with no card data

cur.close()  
cnx.close()

#### Inner joins
Show only rows with matches from both tables

Inner join can be replaced with a left join with where clauses to filter out rows without matches.

In [ ]:
cnx = sqlite3.connect('data/credit_card_data/credit_card_data.sqlite')
cur = cnx.cursor()

# Request: Show me all fraudulent transactions

fraudulent_transactions = pd.read_sql(
    '''SELECT transactions.* 
       FROM transactions 
         INNER JOIN frauds ON transactions.Transaction_ID = frauds.Transaction_ID''',
    con=cnx)
display(fraudulent_transactions)

# Possible next steps: Show all the customers with fraudulent transactions

defrauded_customers = pd.read_sql(
    '''SELECT DISTINCT customers.*
       FROM transactions
         INNER JOIN frauds ON transactions.Transaction_ID = frauds.Transaction_ID
         INNER JOIN cards ON cards.Card_Number = transactions.Credit_Card_ID
         INNER JOIN customers ON customers.Cust_ID = cards.Cust_ID''',
    con=cnx)
display(defrauded_customers)  

   
cur.close()  
cnx.close()